## Presentación

Estos ejercicios tienen el fin de poder evaluar la situación en la que se encuentra cada unx de ustedes a esta altura de la materia. No es un examen, ni llevará ningún tipo de nota.

Les pedimos que respeten al pie de la letra el nombre de las funciones que les damos para completar y su "signature". Esto es, que respeten los argumentos de entrada y la salida. Esto es porque tenemos una forma de verificación automática que de otra manera no funciona. Cuando hayamos recibido los notebooks, haremos público el código de control.

Por supuesto, si quieren agregar celdas y más código, si quieren hacer pruebas para asegurarse de que el código funciona, etc., pueden hacerlo aquí mismo.

La _modalidad de entrega preferencial_ es a través de GitHub:
1. Hacer un fork del [repositorio](https://github.com/exord/UNSAM_IA), tocando el botón de arriba a la derecha (ver figura). 

2. Crear un subdirectorio dentro del directorio <tt>entregas</tt> con el formato "ApellidoNombre"
3. Completar este notebook y colocarlo en ese directorio. Hacer un "commit" y un "push" a su versión del repositorio.
4. Realizar un <tt>pull request</tt> desde GitHub y seguir las instrucciones. Más información en la [página de GitHub](https://help.github.com/en/github/collaborating-with-issues-and-pull-requests/about-pull-requests).

De ser necesario, en caso de sufrir de **alergia severa a GitHub**, el notebook puede enviarse por email.

La **fecha límite** para la entrega es el **martes 21 de abril a las 20 horas**.

<figure>
    <img src="figures/fork_pullrequest.jpeg", alt="Fork", width="800px", 
         style="float: left; margin-right: 10px;"/>
    
     <figcaption>Fig.1 - Ubicación de los botones para Fork y Pull request en Github.</figcaption>
</figure>

## 1. Probabilidad Condicional

In [0]:
#Si estas en Google Colab, ejecuta esta celda una unica vez:
!mkdir datasets
!mkdir datasets/student-alcohol-consumption
!wget https://raw.githubusercontent.com/exord/UNSAM_IA/master/datasets/student-alcohol-consumption/student-mat.csv
!mv student-mat.csv datasets/student-alcohol-consumption/student-mat.csv

--2020-04-17 16:28:49--  https://raw.githubusercontent.com/exord/UNSAM_IA/master/datasets/student-alcohol-consumption/student-mat.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41983 (41K) [text/plain]
Saving to: ‘student-mat.csv’

student-mat.csv     100%[===================>]  41.00K  --.-KB/s    in 0.01s   

2020-04-17 16:28:49 (2.83 MB/s) - ‘student-mat.csv’ saved [41983/41983]



Similar al ***Caso: Apruebo si falto a clase?*** del [notebook de probabilidad condicional](02_Condicional.ipynb), calcule la probabilidad de que un alumno haya faltado menos de 3 veces dado que sacó más de un 60% en la materia, i.e.
$$
P( \text{absences} < 3 | \text{G3} \geq 12).
$$

Por favor, utilice el siguiente signature. Se recomienda cambiar el código solamente entre los comentarios "Start" y "Finish". 
Si quiere probar cosas, siéntase libre de crear nuevas celdas de código, aunque estas no serán evaluadas.

Respuesta:
Para calcular la probabilidad de que un alumno haya faltado menos de 3 veces dado que sacó más de un 60% en la materia se calcula como:

$$
P( \text{absences} < 3 | \text{G3} \geq 12) = \frac{P( \text{G3} \geq 12 \cap \text{absences}< 3)}{P(\text{G3}\geq 12)}
$$


In [0]:
import numpy as np
import pandas as pd

def prob_condicional(
    df=pd.read_csv('datasets/student-alcohol-consumption/student-mat.csv')):
    """Calcula P(absences < 3 | G3 >= 12).

    Input:
    df -- dataset de estudiantes
    Output:
    p        -- probabilidad de que absences > 3 dado que G3 >= 12
    """
    #Start
    #Primero manipulo el dataframe de pandas para sacar los datos que quiero
    datosDelDF = df.copy()

    #Cuento cuantos alummnos tienen mas de un 60% de la materia
    datosDelDF["Mayor de 12"] = np.where(datosDelDF["G3"] >= 12 ,1,0)
    #Cuento aquellos alumnos que tienen mas de 3 ausencias
    datosDelDF["Mas de 3 ausencias"] = np.where(datosDelDF["absences"]>3,1,0)
    #Cuento los alumnos que hay
    datosDelDF["Constante"] = 1


    #Me quedo con los datos que necesito
    datosDelDF = datosDelDF[["Constante","Mayor de 12","Mas de 3 ausencias"]]
    alumnosTotales = datosDelDF["Constante"].sum()

    #Me creo una tabla para clasificar los datos
    datosDelDF = pd.pivot_table(
    datosDelDF,
    values = 'Constante',
    index = ['Mayor de 12'],
    columns = ['Mas de 3 ausencias'],
    aggfunc = np.sum,
    fill_value = 0)

    """Con esto consigo una tabla con los datos. Ahora puedo calcular la prob.
    de tener mas de 60% en la materia y la probabilidad de tener mas de 3 ausencias
    y tener mas de 60% en la materia.
    """
    print(datosDelDF)

    #Calculo los aprobados totales
    aprobados = (datosDelDF.iloc[1][1] + datosDelDF.iloc[1][0])/alumnosTotales
    
    #Caclculo los aprobados con mas de 3 ausencias
    aprobadosYMasDeTresAusencias = datosDelDF.iloc[1][1]/alumnosTotales

    #guarda el valor final, como un número, en esta variable
    resultado = aprobadosYMasDeTresAusencias / aprobados

    #Finish
    return resultado

print("La probabilidad de tener mas de 3 ausencias dado que se tiene mas de 60% en la materia es: {}".format(prob_condicional()))

Mas de 3 ausencias    0    1
Mayor de 12                 
0                   107  126
1                    84   78
La probabilidad de tener mas de 3 ausencias dado que se tiene mas de 60% en la materia es: 0.48148148148148145


## 2. Inferencia Bayesiana

Se tira una moneda N veces, de las cuales X sale cara. La moneda sale cara con una frecuencia $\mu$ desconocida, pero que se cree que es *fair*. 

Construya una función que recibe el vector con los N resultados (1: cara, 0: seca) y con él calcule tanto el posterior del parámetro $\mu$ como la probabilidad de que $\mu>0.7$. Use como prior una distribución normal con media 0.5 y desviación estandar 0.1 ($\mathcal{N}(0.5,0.1)$)

Por favor, utilice el siguiente signature. Se recomienda cambiar el código solamente entre los comentarios "Start" y "Finish". 
Si quiere probar cosas, siéntase libre de crear nuevas celdas de código, aunque estas no serán evaluadas.

<b>Ayuda</b>: usar los métodos <tt>pdf</tt> y <tt>pmf</tt> de las clases de <tt>scipy.stats</tt> que se importan al principio de la celda. Revisar también los notebooks de [teoría](03_Inferencia.ipynb) y [práctica](03p_Inferencia_Soluciones.ipynb) que se presentaron en clase.


In [0]:
import numpy as np
from scipy.stats import binom, norm

def inferencia_moneda(tiradas, seed=123):
    """Realiza inferencia bayesiana en el parametro mu de una moneda.

    Input:
    tiradas -- Resultado de N tiradas
    seed    -- Semilla para np.random (default=123)

    Output:
    p        -- probabilidad de que mu>0.7
    posterior-- posterior de mu, shape=(100,)
    """
    np.random.seed(seed)
    mu, step = np.linspace(0,1,num=100, retstep=True) #step es el espacio entre puntos

    #Numero de tiradas, y de veces que salio cara
    N=len(tiradas)
    X=np.sum(tiradas)

    #Start
    #El prior debe ser una distribución normal con media mu = 0.5 y desvio std = 0.1
    #PAra esto uso la función norm.pdf y le paso el mu, la media y el desvio std
    prior = norm.pdf(mu,0.5,0.1) 
    
    #La verosimilitud en este caso se trata de una binomial. Los exitos serían la cantidad
    #de caras que salieron (guardadas en X)
    verosimilitud = binom.pmf(X,N,mu) #La verosimilitud evaluada en el vector mu

    #Como el posterior es proporcional al producto del prior por la likelihood calculo
    #el posterior hasta que consiga la evidencia
    posterior = prior*verosimilitud #el posterior no normalizado
    
    #La evidencia es la probabilidad marginal, por lo que se debe integrar la likelihood
    #para esto debo sumar todos los puntos de la likelihood multiplicados por el tamaño del paso
    #entre puntos
    evidencia = verosimilitud.sum() * step #evidencia o verosimilitud marginalizada

    #El posterior verdadero es el producto entre la likelihood y el prior sobre la evidencia
    posterior = posterior/evidencia #normalizamos para obtener el posterior

    #La probabilidad de que mu sea mayor a 0.7 es la suma de todas las probabilidades
    #en las que mu es mayor a 0.7, para ello, integro de 0,7 hasta el max

    p = posterior[mu > 0.7].sum() * step #Probabilidad de que mu > 0.7 
    #Finish
    assert len(posterior) == len(mu), "Tamaño del posterior distinto al de mu"
    return p, np.array(posterior)


(0.05944050468072846, array([0.00000000e+00, 3.36708663e-16, 6.82016994e-14, 1.82471626e-12,
       2.11814325e-11, 1.54859821e-10, 8.41864555e-10, 3.71811424e-09,
       1.40648229e-08, 4.71499644e-08, 1.43374284e-07, 4.02079470e-07,
       1.05274766e-06, 2.59745612e-06, 6.08307117e-06, 1.35999055e-05,
       2.91602737e-05, 6.01905823e-05, 1.19978411e-04, 2.31553063e-04,
       4.33640994e-04, 7.89515318e-04, 1.39972948e-03, 2.41985392e-03,
       4.08437144e-03, 6.73776832e-03, 1.08735123e-02, 1.71809640e-02,
       2.65992682e-02, 4.03758926e-02, 6.01257587e-02, 8.78849418e-02,
       1.26150904e-01, 1.77899451e-01, 2.46567424e-01, 3.35989962e-01,
       4.50282391e-01, 5.93659690e-01, 7.70191277e-01, 9.83495282e-01,
       1.23638434e+00, 1.53048338e+00, 1.86584788e+00, 2.24061775e+00,
       2.65074562e+00, 3.08983825e+00, 3.54914544e+00, 4.01772100e+00,
       4.48276711e+00, 4.93015626e+00, 5.34510673e+00, 5.71296984e+00,
       6.02007244e+00, 6.25454826e+00, 6.40708859e+00, 

## 3. Regresión Lineal

Construye una función que dado un input (X_train, Y_train) realice un ajuste lineal de cuadrados mínimos, y con ello devuelva las predicciones para el input X_test como un vector Y_test.

Por favor, utilice el siguiente signature. Se recomienda cambiar el código solamente entre los comentarios "Start" y "Finish". 
Si quiere probar cosas, siéntase libre de crear nuevas celdas de código, aunque estas no serán evaluadas.

<b>Ayuda</b>: pueden usar la implementación de las ecuaciones normales que aparece en el notebook de [modelos lineales](05_ModelosLineales.ipynb) o las clases del módulo <tt>sklearn.linear_model</tt>.

In [0]:
import numpy as np

def ajuste_lineal(X_train, Y_train, X_test):
    """Calcula un ajuste lineal.

    Input:
    X_train -- datos de entrenamiento, shape=(N,1)
    Y_train -- etiquetas de entrenamiento, shape=(N,1)
    X_test  -- datos de testeo, shapes=(N_test,1)

    Output:
    Y_test  -- prediccion de etiquetas para X_test, shape=(N_test,1)
    """

    N=len(X_train)
    N_test=len(X_test)
    #Start

    #Primero me formo mi matriz de diseño, teniendo en cuenta que phi sub cero(x)=1
    phiSubUno = X_train

    #Como se trata de un ajuste lineal con M = 1, tengo una matriz de dos columnas y
    #misma cantidad de filas que mis datos
    matrizPhi = np.hstack(phiSubUno*0.0+1,phiSubUno)
    #Ahora necesito resolver la ecuación que marco en la siguiente caja de texto
    #Para esto necesito el producto de mi matriz phi transpuesta por phi
    phiTporPhi = matrizPhi.T @ matrizPhi

    #Ahora tengo que buscar mi producto phi por mi vector T que en este caso sería
    #los datos de Y_test
    phiPorY = matrizPhi @ Y_train

    #Por último, resuelvo la ecuación para tomar los valores de los parametros omega
    # de maxima verosimilitud

    wMaximaVerosimilitud = np.linalg.solve(phiTporPhi,phiPorY)

    #Con mi vector de máxima verosimilitud, obtengo el Y_test a partir del
    #X_test
    datosDeTest = np.hstack((X_test*0.0+1,X_test))
    #ahora guardamos los resultados en esta variable
    Y_test = np.dot(datosDeTest,wMaximaVerosimilitud)
    #Finish
    assert len(Y_test) == len(X_test), "Tamaño de Y_test distinto al de X_test"
    return np.array(Y_test)

$$\begin{pmatrix}
\omega_0^\text{ML}\\
\omega_1^\text{ML}\\
\end{pmatrix} 
= \left(\Phi^T \Phi\right)^{-1}\Phi^T \,\vec{t}
$$

Que es lo mismo que:
$$(\Phi^T\Phi)W_{ML} = \Phi\vec{t} $$